In [68]:
from datascience import *
from pandas import read_stata
import numpy as np
from pygrowup import Calculator

import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)


In [69]:
#from lab09_minimize import *

def minimize_multi(f, start=None, **vargs):
  def expanded_f(*args):
    return f(args)
  return minimize(expanded_f, start=start, method="L-BFGS-B", **vargs)

In [70]:
data = Table.read_table('Lab_3.csv')
data

Household ID,Individual ID,Day of Birth,Month of Birth,Birth Year,Age in Years,Sex,Height,Weight,Relationship to HH Head,Mother ID,Father ID,Currently Enrolled in School,Years of Schooling - Level,Years of Schooling - Years,Master ID,Day of Interview,Month of Interview,Year of Interview,Days Old,Years Old,Months Old,z_scores,Rounded Months
20111642,3,3,7,1976,31,2,1,-99,9,-99,-99,-99,2,5,-1363194477,10,7,2007,11329.8,31.0404,59,-22.86,59
20111642,4,10,9,1995,11,2,1,-99,5,3,2,1,-99,-99,-1363194476,10,7,2007,4322,11.8411,59,-22.86,59
20111642,5,20,4,1997,10,1,1,-99,5,3,2,1,-99,-99,-1363194475,10,7,2007,3734,10.2301,59,-23.58,59
20111642,6,26,12,1999,7,1,1,-99,5,3,2,-99,-99,-99,-1363194474,10,7,2007,2753.5,7.54384,59,-23.58,59
20111642,7,17,1,2000,7,1,1,-99,5,3,2,-99,-99,-99,-1363194473,10,7,2007,2732.75,7.48699,59,-23.58,59
20111642,8,26,5,2002,5,1,105.9,22.6,5,3,2,-99,-99,-99,-1363194472,10,7,2007,1871.25,5.12671,59,-0.76,59
20111642,9,5,1,2005,2,2,80.5,9.5,5,3,2,-99,-99,-99,-1363194471,10,7,2007,918.5,2.51644,30.1148,-2.88,30
20111642,10,12,4,2006,1,2,70.3,7.1,5,3,2,-99,-99,-99,-1363194470,10,7,2007,454.75,1.24589,14.9098,-2.27,15
20113642,1,13,4,1966,41,1,1,-99,1,-99,-99,-99,-99,-99,-1361194479,10,7,2007,15063.8,41.2705,59,-23.58,59
20113642,2,1,1,1969,38,2,1,-99,2,-99,-99,-99,-99,-99,-1361194478,10,7,2007,14071.5,38.5521,59,-22.86,59


## Multiple Regression

<font color="Blue"> Another way to predict outcomes is with multivariate regression.  Prepare two multivariate regressions to compare: one regression will have all the variables you think are important and the other will have variables you think are not important.  Compare the R^2 of these regressions.  Were you right? Explain.

We will look at the steps to building multivariate regression examining how age and sex predict height.

In [71]:
# Make sure all your data is in integer format and keep only observations with height, age & sex data
data["Height"]=data.apply(int, 'Height')
data["Age in Years"]=data.apply(int, 'Age in Years')
data["Sex"]=data.apply(int, 'Sex')
# Make a column that will be associated with the intercept
data["For Intercept"]=1

# Select all the variables you want to analyze including intercept and outcome.
# (Until we have the correct minimizing function, just include one variable and the outcome.)
data=data.select("Height","Age in Years","Sex","For Intercept")
data=data.where(data['Height']>10)
data

Height,Age in Years,Sex,For Intercept
105,5,1,1
80,2,2,1
70,1,2,1
113,5,1,1
89,3,1,1
74,1,2,1
89,3,1,1
87,2,1,1
112,5,1,1
85,3,2,1


In [72]:
# Divide the table in two: predictor variables (sex & age) & outcome variable (height)
X_features_table =data.drop("Height")
X_true_values = data.column("Height") # note: this is an array
X_features_table

Age in Years,Sex,For Intercept
5,1,1
2,2,1
1,2,1
5,1,1
3,1,1
1,2,1
3,1,1
2,1,1
5,1,1
3,2,1


In [73]:
# Choose some intial values for maximizing the coefficients 
# An easy choice is all 0's.  
# There will be the same number of coefficients as there are predictors - a slope for each predictor.
X_initial_coefficient_guess = np.zeros(X_features_table.num_columns)
X_initial_coefficient_guess

array([ 0.,  0.,  0.])

In [74]:
def predict_all(features_table, coefficients):
    """
    Given a table of features called features_table and some coefficients,
    produces linear predictions for each row of features_table.
    
    features_table should be a table with one column for each feature
    being used to predict.  Each row represents a house in the task
    we're doing in this lab.
    
    coefficients should be an array with one element for each column in
    features_table, like the coefficients computed by the function
    least_squares_coefficients.
    
    Returns an array of predictions, one for each row of features_table.
    
    For example, in the house price prediction task we're working on in
    this lab, each row of data is the features of one house sale, like
    the number of bedrooms or the size of the house.  To make a
    prediction for the price of a house, we multiply each of these
    features by the corresponding coefficient in the coefficients
    array, then sum the results.
    
    An even more detailed example: Suppose we have just one house whose
    price we want to predict, and two features, Bedrooms and Size.
    Then features_table will have 1 row and 2 columns, Bedrooms and Size.
    Say their values are 2 and 1500.  The coefficients array will have 2
    numbers in it, say -1000 and 200.  Our prediction for this house is:

        features_table.column(0).item(0)*coefficients.item(0) + \
        data.column(1).item(0)*coefficients.item(1)
    
    or
    
        2*-1000 + 1500*200
    
    which is $298,000.  So we will return an array with just one number
    in it, and it will look like:
    
        np.array([298000])
    
    If there were 3 rows in features_table, we would return a 3-element
    array instead, containing the predicted prices for each row.
    """
   # assert features_table.num_columns == len(coefficients), /
    """
    The first argument to predict_all should be a table with one
    column for each feature.  That means it should have the same
    number of columns as the coefficients array (the second
    argument) has elements.
    """
    def predict(features):
        # Given an array of features, produce one prediction.
        return sum(features * coefficients)
    predictions = Table().with_column('features', features_table.rows).apply(predict, 'features')
    return predictions

In [75]:
X_predictions=predict_all(X_features_table, X_initial_coefficient_guess)
X_predictions

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [76]:
# change the initial coefficient guess up above to make a different prediction!
X_new_coefficient_guess = [1]
X_predictions=predict_all(X_features_table, X_new_coefficient_guess)
X_predictions

array([7, 5, 4, ..., 3, 4, 3])

In [77]:
def compute_errors(features_table, coefficients, true_values):
    """
    Computes the prediction errors for a linear model with the given
    coefficients when predicting the true values for the given
    examples.
    
    features_table should be a table with one column for each feature
    being used to predict.  Each row represents a house in the task
    we're doing in this lab.
    
    coefficients should be an array of numbers, one for each feature.
    
    true_values should be an array of numbers, one for each row in
    features_table.  It records the true prices of each house.
    """
    return predict_all(features_table, coefficients) - true_values



In [78]:
X_errors=compute_errors(X_features_table, X_initial_coefficient_guess, X_true_values)
X_errors

array([-105.,  -80.,  -70., ...,  -69.,  -83.,  -74.])

In [79]:
def rmse(errors):
    """
    Computes the root mean squared error when a regression model makes
    the given errors.  So errors should be an array of numbers, one for
    each row in some data table for which we're computing predictions
    (that is, each house).  Each number is the prediction error of some
    regression model (when predicting the price of a house).
    """
    return np.mean(errors**2)**0.5



In [80]:
r2=rmse(X_errors)
r2

88.002087362612187

In [81]:
def make_least_squares_objective_function(features_table, true_values):
    """
    Makes an objective function for training data in the features_table
    table, where the true values we're trying to predict are true_values.
    
    features_table should be a table with one column for each feature
    being used to predict.  Each row represents a house in the task
    we're doing in this lab.
    
    true_values should be an array of numbers, one for each row in
    features_table.  It records the true prices of each house.
    
    The returned value is a function.  That function takes an array of
    coefficients and returns a number.  Larger values of that number
    mean that those coefficients produce worse prediction errors.
    """
    def objective_function(coefficients):
        errors = compute_errors(features_table, np.array(coefficients), true_values)
        return rmse(errors)
    return objective_function



In [82]:
X_objective_function = make_least_squares_objective_function(X_features_table, X_true_values)
X_objective_function

<function __main__.make_least_squares_objective_function.<locals>.objective_function>

In [83]:
def least_squares_coefficients(training_data, predicted_column_name):
    """
    Performs multiple linear regression predicting predicted_column_name
    using the other columns of training_data as features.
    
    training_data should be a table with one column for each feature
    being used to predict, plus one column for the value we're trying
    to predict.  That column's name should equal predicted_column_name.
    Each row represents a house in the task we're doing in this lab.
    
    predicted_column_name should be a string, the name of the column in
    training_data that we're trying to predict.
    
    Returns an array of coefficients, one for each feature (that is, one
    for each column in training_data other than predicted_column_name).
    
    For example, if training_data has 3 columns, Bedroom, Size, and Price,
    and predicted_column_name is "Price", then we will use Bedroom and
    Size to predict Price.  This function will return an array of 2
    numbers, a regression coefficient for Bedroom (like -1000) and a
    regression coefficient for Size (like 200).
    """
    features_table = training_data.drop(predicted_column_name)
    true_values = training_data.column(predicted_column_name)
    objective_function_OLS = make_least_squares_objective_function(features_table, true_values)
    
    # Now we find the coefficients that produce the smallest
    # error.
    initial_coefficient_guess = np.zeros(features_table.num_columns)
    best_coefficients = minimize_multi(objective_function_OLS, start=initial_coefficient_guess)
    if features_table.num_columns == 1:
        return np.array([best_coefficients])
    else:
        return best_coefficients

In [84]:
X_best_coefficients = minimize_multi(X_objective_function, start=X_initial_coefficient_guess)
X_best_coefficients

array([  7.83037917,  -0.74819884,  67.83253874])

In [85]:
My_Coefficients = least_squares_coefficients(data, 'Height')
My_Coefficients

array([  7.83037917,  -0.74819884,  67.83253874])

Let's see how well our model fits the data.  We will calculate the R^2.

In [86]:
mean_height=np.average(data["Height"])
mean_height

86.748953001396004

In [87]:
My_predictions=predict_all(X_features_table, My_Coefficients)
data["Predicted"]=My_predictions
data["Diff_Predict_SQ"]=(data["Height"]-data["Predicted"])**2
data["Diff_True_SQ"]=(data["Height"]-mean_height)**2
data

Height,Age in Years,Sex,For Intercept,Predicted,Diff_Predict_SQ,Diff_True_SQ
105,5,1,1,106.236,1.52828,333.101
80,2,2,1,81.9969,3.98761,45.5484
70,1,2,1,74.1665,17.3599,280.527
113,5,1,1,106.236,45.7485,689.117
89,3,1,1,90.5755,2.48213,5.06721
74,1,2,1,74.1665,0.027729,162.536
89,3,1,1,90.5755,2.48213,5.06721
87,2,1,1,82.7451,18.1042,0.0630246
112,5,1,1,106.236,33.221,637.615
85,3,2,1,89.8273,23.3026,3.05884


In [88]:
r_squared=1-np.sum(data["Diff_Predict_SQ"])/np.sum(data["Diff_True_SQ"])
r_squared

0.79364575957588268

<font color="Blue"> Item 1: Let's be more systematic about building the model - we will add variables to try to get the largest R^2.  Adding more covariates will improve the fit of the model, but we are also making the model more complex.  Let's add covariates in order of importance, keeping those that increase the adjusted R^2.  The formula for the adjusted R^2 is 1-(1-R^2)(N-1)/(N-p-1), where N= sample size and p= number of predictors.

This model is of correlation, not causation.  But since this is a first indication of a potential causal relationship, we may as well try a policy and then later see if it works out. <font color="Blue"> Item 2: If you were attempting to change the outcome based on changing the feature, which feature would you try to change?  Don't just consider the one with the highest correlation, but also take into account costs and difficulty of changing the feature. (Not a data exercise)

<font color="Blue"> Item 3: Now trade coefficients with someone who is working with data from another country.  How well did their model fit your data: what is the accuracy of their model across borders? Comment on what this implies for policy.

<font color="Blue"> 